In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import random

# 1. Define DAG and logical strain values
G = nx.DiGraph()

# Logical states with D(s)
states = {
    "s_Bell": 1.0,
    "s_mid": 0.5,
    "s_01": 0.0,
    "s_10": 0.0
}

# Add nodes with attributes
for s, d in states.items():
    G.add_node(s, D=d)

# Add directed edges: only to equal or lower strain
for si in G.nodes:
    for sj in G.nodes:
        if si != sj and states[sj] <= states[si]:
            G.add_edge(si, sj)

# 2. Deterministic collapse
def collapse_path(G, start, D_crit=0.0):
    path = [start]
    current = start
    while G.nodes[current]["D"] > D_crit:
        neighbors = list(G.successors(current))
        if not neighbors:
            break
        next_node = min(neighbors, key=lambda n: G.nodes[n]["D"])
        if G.nodes[next_node]["D"] >= G.nodes[current]["D"]:
            break
        path.append(next_node)
        current = next_node
    return path

# 3. Stochastic collapse with soft logic
def stochastic_collapse_path(G, start, beta=5.0, D_crit=0.0):
    path = [start]
    current = start
    while G.nodes[current]["D"] > D_crit:
        neighbors = list(G.successors(current))
        if not neighbors:
            break
        D_vals = np.array([G.nodes[n]["D"] for n in neighbors])
        weights = np.exp(-beta * D_vals)
        probs = weights / weights.sum()
        next_node = np.random.choice(neighbors, p=probs)
        if G.nodes[next_node]["D"] >= G.nodes[current]["D"]:
            break
        path.append(next_node)
        current = next_node
    return path

# 4. Visualization
def visualize_graph(G, path=None):
    pos = nx.spring_layout(G, seed=42)
    labels = {n: f"{n}\nD={G.nodes[n]['D']}" for n in G.nodes}
    nx.draw(G, pos, with_labels=True, labels=labels, node_size=2500, node_color='lightblue', font_size=9, font_weight='bold')
    if path:
        edge_path = list(zip(path[:-1], path[1:]))
        nx.draw_networkx_edges(G, pos, edgelist=edge_path, edge_color='red', width=2)
    plt.title("Logical Collapse DAG")
    plt.axis('off')
    plt.show()

# Run both simulations and visualize
det_path = collapse_path(G, "s_Bell")
stoch_path = stochastic_collapse_path(G, "s_Bell", beta=5.0)
print("Deterministic Path:", det_path)
print("Stochastic Path:", stoch_path)

# Visualize deterministic path
visualize_graph(G, det_path)